## 최종코드

In [99]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대 팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)와 같아야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        if all(games == rounds_target for games in player_games.values()) and len(matches) == len(players):
            return matches

# ===============================
# 2. leftover 경기(G코트) 배정을 위한 함수 (인접 슬롯 조건 제거)
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 현재까지 배정된 G코트 결과 (길이 12, "경기 없음" 또는 경기 튜플)
    
    G코트에서는 인접 슬롯 조건은 완화하고, 오직 slot 자체에서 메인 스케줄 및 extra_assignment와 충돌 없는지 검사.
    """
    players_extra = get_players(extra_match)
    if get_players(main_schedule[slot]) & players_extra:
        return False
    if extra_assignment[slot] != "경기 없음":
        if get_players(extra_assignment[slot]) & players_extra:
            return False
    return True

def assign_extra_schedule_for_group(extras, main_schedule, max_attempts=100):
    """
    extras: 리스트 leftover 경기 (튜플들)
    main_schedule: 메인 스케줄 (길이 12)
    재시도하면서, extras를 12시간대 내에 충돌 없이 배정할 수 있는지 랜덤 재조정.
    모든 extras가 배정되면 배정 결과(길이 12 리스트)를 반환, 불가능하면 None 반환.
    """
    for attempt in range(max_attempts):
        assignment = ["경기 없음"] * 12
        extras_shuffled = extras[:]  # 복사 후 재조정
        random.shuffle(extras_shuffled)
        slots = list(range(12))
        random.shuffle(slots)
        assigned_count = 0
        for extra in extras_shuffled:
            placed = False
            for slot in slots:
                if assignment[slot] == "경기 없음":
                    if can_place_extra_in_slot(extra, slot, main_schedule, assignment):
                        assignment[slot] = extra
                        assigned_count += 1
                        placed = True
                        break
            if not placed:
                break  # 이 시도에서는 모든 extras 배정 실패
        if assigned_count == len(extras):
            return assignment
    return None

# ===============================
# 3. 최종 스케줄 생성 및 코트 배정 (시간대 1~12)
# ===============================

def main(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4):
    """
    player_groups: dict, 그룹 이름 -> 선수 리스트
    target_rounds: 사용 가능한 시간대 수 (여기서는 12)
    max_global_attempts: 글로벌 재시도 최대 횟수
    rounds_target: 각 선수가 해야 하는 경기 수 (여기서는 4)
    
    조건:
      - 각 선수는 정확히 rounds_target 경기 (4경기)를 진행
      - 각 그룹별로 생성된 경기 수 = 그룹 인원수
      - 메인 스케줄: 생성 경기 중 처음 target_rounds (1~12경기)
      - leftover(13경기 이상)는 G코트에 배정, 단 한 슬롯에 한 경기만 배정하며,
        우선순위 순으로 배정 (그리고 8번째 시간대(시간=8, 인덱스 7)는 "이벤트 시간"으로 처리)
    """
    # (가) 각 그룹별 경기 생성
    group_results = {}
    for group_name, players in player_groups.items():
        group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=rounds_target)
    
    # (나) 메인 스케줄과 leftover 분리
    main_schedules = {}
    extra_schedules = {}
    for group_name, matches in group_results.items():
        main_schedules[group_name] = matches[:target_rounds]   # 인덱스 0~11 => 시간대 1~12
        extra_schedules[group_name] = matches[target_rounds:]    # leftover 경기 (있으면)
    
    # (다) 각 그룹별로 leftover 경기 배정 (G코트 배정)
    group_extra_assignment = {}
    all_assigned = True
    for group, extras in extra_schedules.items():
        assignment = assign_extra_schedule_for_group(extras, main_schedules[group], max_attempts=100)
        if assignment is None:
            all_assigned = False
            break
        group_extra_assignment[group] = assignment
    if not all_assigned:
        return None  # 글로벌 재시도 실패
    
    # (라) 최종 G코트 스케줄: 각 시간대에 단 하나의 leftover 경기만 배정 (우선순위 순)
    # 우선순위: ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    final_G_schedule = ["경기 없음"] * target_rounds
    for slot in range(target_rounds):
        # 8번째 시간대 (시간=8, 인덱스 7)는 강제로 "이벤트 시간" 처리
        if slot == 7:
            final_G_schedule[slot] = "이벤트 시간"
            continue
        for group in group_order:
            extra_assign = group_extra_assignment.get(group, ["경기 없음"] * target_rounds)
            if extra_assign[slot] != "경기 없음":
                team1, team2 = extra_assign[slot]
                final_G_schedule[slot] = f"{', '.join(team1)} vs {', '.join(team2)}"
                break  # 한 슬롯에는 한 경기만 배정
    
    # (마) A~F 코트 배정 (메인 스케줄)
    # 코트 매핑:
    # A코트: 여자 그룹 3, B코트: 여자 그룹 2, C코트: 여자 그룹 1,
    # D코트: 남자 그룹 3, E코트: 남자 그룹 2, F코트: 남자 그룹 1
    court_map = {
        "A코트": "여자 그룹 3",
        "B코트": "여자 그룹 2",
        "C코트": "여자 그룹 1",
        "D코트": "남자 그룹 3",
        "E코트": "남자 그룹 2",
        "F코트": "남자 그룹 1"
    }
    schedule_df = pd.DataFrame(columns=["시간"] + list(court_map.keys()) + ["G코트"])
    schedule_df["시간"] = range(1, target_rounds+1)
    
    for round_idx in range(target_rounds):
        for court, group in court_map.items():
            if group in main_schedules and round_idx < len(main_schedules[group]):
                match = main_schedules[group][round_idx]
                if match != "경기 없음":
                    team1, team2 = match
                    schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
                else:
                    schedule_df.at[round_idx, court] = "경기 없음"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]
    
    schedule_df.fillna("경기 없음", inplace=True)
    return schedule_df

# ===============================
# 4. 체크 함수: 각 선수 경기 수 및 동일 시간대 중복 검사
# ===============================

def parse_match(match_str):
    """
    match_str: "A vs B" 형태의 문자열 (또는 "경기 없음", "이벤트 시간")
    반환: 해당 경기에서 출전한 선수의 집합.
    """
    if match_str in ["경기 없음", "이벤트 시간"]:
        return set()
    try:
        teams = match_str.split(" vs ")
        team1 = [x.strip() for x in teams[0].split(",")]
        team2 = [x.strip() for x in teams[1].split(",")]
        return set(team1 + team2)
    except Exception as e:
        return set()

def count_games_per_player(schedule_df):
    """
    schedule_df: 최종 일정표 DataFrame (시간, A코트, B코트, ..., G코트)
    각 선수별 출전 횟수를 계산하여 dict로 반환.
    """
    counts = {}
    for idx, row in schedule_df.iterrows():
        for col in schedule_df.columns:
            if col == "시간":
                continue
            players = parse_match(row[col])
            for player in players:
                counts[player] = counts.get(player, 0) + 1
    return counts

def check_same_time_slot(schedule_df):
    """
    schedule_df: 최종 일정표 DataFrame (시간, A코트, ..., G코트)
    각 시간대(행)에서 여러 코트에 걸쳐 동일한 선수가 출전한 경우를 찾아 dict로 반환.
    반환 형식: {시간대: {선수: 중복 횟수, ...}, ...}
    """
    conflicts = {}
    for idx, row in schedule_df.iterrows():
        time_slot = row["시간"]
        all_players = []
        for col in schedule_df.columns:
            if col == "시간":
                continue
            all_players.extend(list(parse_match(row[col])))
        counter = {}
        for player in all_players:
            counter[player] = counter.get(player, 0) + 1
        slot_conflicts = {player: cnt for player, cnt in counter.items() if cnt > 1}
        if slot_conflicts:
            conflicts[time_slot] = slot_conflicts
    return conflicts

# ===============================
# 5. 글로벌 재시도: 모든 선수의 경기 수가 4회가 될 때까지 재생성
# ===============================

if __name__ == '__main__':
    player_groups = {
        "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
        "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
        "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
        "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
        "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
        "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"]
    }
    
    max_global_attempts = 1000
    final_schedule_df = None
    for attempt in range(max_global_attempts):
        schedule_df = main(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4)
        if schedule_df is None:
            continue
        game_counts = count_games_per_player(schedule_df)
        # 모든 선수가 정확히 4경기인지 확인
        if all(count == 4 for count in game_counts.values()):
            final_schedule_df = schedule_df
            break

    if final_schedule_df is not None:
        display(final_schedule_df)
        print("\n각 선수별 출전 횟수:")
        game_counts = count_games_per_player(final_schedule_df)
        for player, cnt in sorted(game_counts.items()):
            print(f"{player}: {cnt}회")
        
        conflicts = check_same_time_slot(final_schedule_df)
        if conflicts:
            print("\n동일 시간대에 중복 출전한 선수:")
            for time_slot, conf in conflicts.items():
                conflict_str = ", ".join([f"{p}({cnt}회)" for p, cnt in conf.items()])
                print(f"시간대 {time_slot}: {conflict_str}")
        else:
            print("\n동일 시간대에 중복 출전한 선수는 없습니다.")
    else:
        print("충분한 충돌 없는 배정 결과를 얻지 못했습니다.")


,시간,A코트,B코트,C코트,D코트,E코트,F코트,G코트
0,1,"문문, 우당탕 vs 린다, 아롬","이피, 꼬맹 vs 지니, 파란하늘","가린, 다미 vs 유하, 연주","김군, 제이슨 vs 쿨쿨, 황더러","미키찬, 페더러 vs Ace, DY","동환, 소니 vs 태보, Bdup","뽐, 수임이 vs 제시, 레이첼"
1,2,"김지현, 야금 vs 크림, Rent","젤로, 사리 vs 수임이, 강진","꾸꾸, 레일라 vs 미아, 커피믹스","성일월드, 스테판 vs 터보, TG","로센, 브라운 vs 푸마, 태규","비케이, 정현 vs 주방, 프리마","밤송이, 성현 vs 재근, 토니"
2,3,"룰루, 반이 vs 클레어, 린다","민크, 제인 vs 제시, 파란하늘","리디아, 럽테닛 vs Hailey, 연주","김군, 우디 vs 쥬니혀니, SJ","매머드, 미키찬 vs 안씨, DY","성현, 소니 vs 큐, 태보","지호, 현민 vs Ace, 태규"
3,4,"문문, 야금 vs 우당탕, Rent","젤로, 꼬맹 vs 지니, 레이첼","기쁨, 레일라 vs 미아, 밀리","마크, 성일월드 vs 스테판, 제이슨","로센, 명자 vs 영훈, Ace","밤송이, 주방 vs Bdup, 프리마",경기 없음
4,5,"반이, 크림 vs 클레어, 아롬","민크, 강진 vs 이피, 제시","가린, 꾸꾸 vs 커피믹스, Hailey","김군, 쥬니혀니 vs 황더러, TG","안씨, 전 vs 페더러, 현민","비케이, 성현 vs 정현, 태보","스테판, 우디 vs 이언짱, 쿨쿨"
5,6,"김지현, 룰루 vs 야금, Rent","뽐, 사리 vs 레이첼, 파란하늘","리디아, 다미 vs 밀리, 연주","이언짱, 제이슨 vs 쿨쿨, 터보","명자, 매머드 vs 지호, DY","매버릭, 소니 vs 프리마, 토니","로저, 마크 vs 성일월드, TG"
6,7,"문문, 반이 vs 크림, 린다","젤로, 수임이 vs 강진, 제인","가린, 기쁨 vs 레일라, 유하","김군, 로저 vs 우디, SJ","로센, 페더러 vs 현민, 태규","동환, 밤송이 vs 비케이, 재근","매버릭, 정현 vs 큐, Bdup"
7,8,"룰루, 야금 vs 우당탕, 클레어","민크, 이피 vs 꼬맹, 파란하늘","꾸꾸, 다미 vs 미아, Hailey","마크, 스테판 vs 이언짱, 쥬니혀니","안씨, 영훈 vs 지호, 푸마","성현, 주방 vs 큐, 토니",이벤트 시간
8,9,"김지현, 반이 vs 크림, 아롬","젤로, 뽐 vs 수임이, 지니","밀리, 럽테닛 vs 커피믹스, 연주","로저, 성일월드 vs 제이슨, SJ","미키찬, 브라운 vs 전, 페더러","매버릭, 재근 vs 태보, 프리마",경기 없음
9,10,"룰루, 클레어 vs Rent, 린다","사리, 이피 vs 제인, 레이첼","기쁨, 다미 vs 미아, 유하","우디, 쥬니혀니 vs 터보, 황더러","로센, 매머드 vs 안씨, 지호","동환, 비케이 vs 소니, 큐","명자, 전 vs 푸마, DY"



각 선수별 출전 횟수:
Ace: 4회
Bdup: 4회
DY: 4회
Hailey: 4회
Rent: 4회
SJ: 4회
TG: 4회
가린: 4회
강진: 4회
기쁨: 4회
김군: 4회
김지현: 4회
꼬맹: 4회
꾸꾸: 4회
다미: 4회
동환: 4회
럽테닛: 4회
레이첼: 4회
레일라: 4회
로센: 4회
로저: 4회
룰루: 4회
리디아: 4회
린다: 4회
마크: 4회
매머드: 4회
매버릭: 4회
명자: 4회
문문: 4회
미아: 4회
미키찬: 4회
민크: 4회
밀리: 4회
반이: 4회
밤송이: 4회
브라운: 4회
비케이: 4회
뽐: 4회
사리: 4회
성일월드: 4회
성현: 4회
소니: 4회
수임이: 4회
스테판: 4회
아롬: 4회
안씨: 4회
야금: 4회
연주: 4회
영훈: 4회
우당탕: 4회
우디: 4회
유하: 4회
이언짱: 4회
이피: 4회
재근: 4회
전: 4회
정현: 4회
제시: 4회
제이슨: 4회
제인: 4회
젤로: 4회
주방: 4회
쥬니혀니: 4회
지니: 4회
지호: 4회
커피믹스: 4회
쿨쿨: 4회
큐: 4회
크림: 4회
클레어: 4회
태규: 4회
태보: 4회
터보: 4회
토니: 4회
파란하늘: 4회
페더러: 4회
푸마: 4회
프리마: 4회
현민: 4회
황더러: 4회

동일 시간대에 중복 출전한 선수는 없습니다.


## 단순 매칭생성하기기

In [21]:
import itertools
import random
import pandas as pd

def initialize_data(players):
    """ 선수 목록 및 초기 데이터 설정 """
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """ 같은 팀에서 1번만 만날 수 있도록 검사 """
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """ 상대 팀으로 2번까지 만날 수 있도록 검사 """
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """ 매치를 추가하고 기록 업데이트 """
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players):
    """ 경기 스케줄 생성 """
    
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < 4 for player in combination) and 
                can_play_together(team1, player_pairs) and 
                can_play_together(team2, player_pairs) and 
                can_play_against(team1, team2, opponent_pairs) and
                not any(player in last_match_players for player in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
                
            if all(games == 4 for games in player_games.values()):
                break
        
        if len(matches) == len(players):
            return matches, player_games, attempt_count

def print_results(matches, player_games, players, attempt_count, group_name):
    """ 경기 결과 및 통계 출력 """
    print(f"\n=== {group_name} 경기 결과 ===")
    for i, match in enumerate(matches):
        team1, team2 = match
        print(f"Match {i + 1}: {', '.join(team1)} vs {', '.join(team2)}")
    print("\n각 선수의 경기 횟수:")
    for player, games in player_games.items():
        print(f"{player}: {games}경기")
    match_count_table = pd.DataFrame(0, index=players, columns=players)
    for team1, team2 in matches:
        for p1 in team1:
            for p2 in team2:
                match_count_table.at[p1, p2] += 1
                match_count_table.at[p2, p1] += 1
    print("\n경기에서 만난 횟수 테이블:")
    print(match_count_table)
    print(f"\n경기 생성에 성공한 시도 횟수: {attempt_count}")

# 선수 그룹 정의
player_groups = {
    "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"],
    "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
    "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
    "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
    "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
    "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"]
}

# random seed
random.seed(40)

# 각 그룹에 대해 경기 생성 및 결과 출력
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    print_results(matches, player_games, players, attempt_count, group_name)



=== 남자 그룹 1 경기 결과 ===
Match 1: 성현, 소니 vs 재근, 큐
Match 2: 매버릭, 비케이 vs Bdup, 프리마
Match 3: 밤송이, 소니 vs 큐, 토니
Match 4: 동환, 매버릭 vs 비케이, 태보
Match 5: 재근, 정현 vs 주방, 프리마
Match 6: 매버릭, 밤송이 vs 태보, Bdup
Match 7: 주방, 큐 vs 프리마, 토니
Match 8: 동환, 밤송이 vs 성현, 정현
Match 9: 비케이, 소니 vs 주방, 태보
Match 10: 동환, 재근 vs 큐, Bdup
Match 11: 매버릭, 소니 vs 정현, 토니
Match 12: 성현, 재근 vs 주방, Bdup
Match 13: 밤송이, 비케이 vs 정현, 프리마
Match 14: 동환, 성현 vs 태보, 토니

각 선수의 경기 횟수:
동환: 4경기
매버릭: 4경기
밤송이: 4경기
비케이: 4경기
성현: 4경기
소니: 4경기
재근: 4경기
정현: 4경기
주방: 4경기
큐: 4경기
태보: 4경기
Bdup: 4경기
프리마: 4경기
토니: 4경기

경기에서 만난 횟수 테이블:
      동환  매버릭  밤송이  비케이  성현  소니  재근  정현  주방  큐  태보  Bdup  프리마  토니
동환     0    0    0    1   1   0   0   1   0  1   2     1    0   1
매버릭    0    0    0    1   0   0   0   1   0  0   2     2    1   1
밤송이    0    0    0    0   1   0   0   2   0  1   1     1    1   1
비케이    1    1    0    0   0   0   0   1   1  0   1     1    2   0
성현     1    0    1    0   0   0   1   0   1  1   1     1    0   1
소니     0    0    0    0   0   0   1   1   1 

## 경기 순서대로 보기좋게 출력하기기 

In [22]:
import pandas as pd

# 각 그룹의 경기 결과 저장
group_results = {}
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    group_results[group_name] = matches[:14]  # 최대 14경기 저장

# 경기 일정 테이블 생성
num_rounds = 14
columns = ["시간", "A코트 (여자 그룹 3)", "B코트 (여자 그룹 2)", "C코트 (여자 그룹 1)",
           "D코트 (남자 그룹 3)", "E코트 (남자 그룹 2)", "F코트 (남자 그룹 1)", "G코트"]
schedule_df = pd.DataFrame(columns=columns)
schedule_df["시간"] = range(1, num_rounds + 1)

group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]

i = 0
for round_num in range(num_rounds):
    for col, group_name in zip(columns[1:], group_order):
        if group_name in group_results and i < len(group_results[group_name]):
            match = group_results[group_name][i]
            team1, team2 = match[0], match[1]
            schedule_df.at[round_num, col] = f"{', '.join(team1)} vs {', '.join(team2)}"
    i += 1  # 순서대로 그룹마다 경기 추가

# 빈 값이 있는 경우 "경기 없음"으로 채우기
schedule_df.fillna("경기 없음", inplace=True)

# 결과 출력
print(schedule_df)


    시간        A코트 (여자 그룹 3)        B코트 (여자 그룹 2)           C코트 (여자 그룹 1)  \
0    1  반이, 크림 vs 클레어, Rent     젤로, 뽐 vs 제인, 레이첼   가린, 다미 vs 럽테닛, Hailey   
1    2    김지현, 룰루 vs 문문, 아롬   강진, 이피 vs 제시, 파란하늘       리디아, 꾸꾸 vs 미아, 유하   
2    3  반이, 야금 vs 우당탕, Rent     뽐, 사리 vs 지니, 레이첼      가린, 기쁨 vs 커피믹스, 연주   
3    4     문문, 크림 vs 린다, 아롬     젤로, 민크 vs 강진, 꼬맹     리디아, 레일라 vs 밀리, 럽테닛   
4    5   김지현, 반이 vs 야금, 우당탕  이피, 지니 vs 레이첼, 파란하늘      기쁨, 꾸꾸 vs 유하, 커피믹스   
5    6  룰루, 클레어 vs Rent, 아롬    사리, 수임이 vs 꼬맹, 제시       미아, 밀리 vs 럽테닛, 연주   
6    7   김지현, 우당탕 vs 크림, 린다    민크, 뽐 vs 지니, 파란하늘      가린, 리디아 vs 다미, 레일라   
7    8    문문, 야금 vs 클레어, 아롬    젤로, 강진 vs 이피, 레이첼  꾸꾸, 밀리 vs 커피믹스, Hailey   
8    9  룰루, 우당탕 vs Rent, 린다    민크, 사리 vs 수임이, 제인      레일라, 미아 vs 유하, 럽테닛   
9   10   김지현, 야금 vs 크림, 클레어     이피, 꼬맹 vs 제시, 지니  꾸꾸, 커피믹스 vs Hailey, 연주   
10  11     룰루, 문문 vs 반이, 린다     뽐, 수임이 vs 강진, 제인       기쁨, 리디아 vs 다미, 미아   
11  12                경기 없음   젤로, 사리 vs 꼬맹, 파란하늘    가린, 밀리 vs 유하, Hailey   
12  13      

## 코트의 선수별로 칼럼나눠서 출력

In [23]:
import pandas as pd

# 각 그룹의 경기 결과 저장
group_results = {}
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    group_results[group_name] = matches[:14]  # 최대 14경기 저장

# 경기 일정 테이블 생성
num_rounds = 14
columns = ["시간"]
group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]

# 각 그룹별 4개 칼럼 생성
for group_name in group_order:
    columns.extend([f"{group_name}_1", f"{group_name}_2", f"{group_name}_3", f"{group_name}_4"])

schedule_df = pd.DataFrame(columns=columns)
schedule_df["시간"] = range(1, num_rounds + 1)

i = 0
for round_num in range(num_rounds):
    for group_name in group_order:
        if group_name in group_results and i < len(group_results[group_name]):
            match = group_results[group_name][i]
            team1, team2 = match[0], match[1]
            players_list = team1 + team2
            for j in range(4):
                schedule_df.at[round_num, f"{group_name}_{j+1}"] = players_list[j]  # 각 칼럼에 한 명씩 배정
    i += 1  # 순서대로 그룹마다 경기 추가

# 빈 값이 있는 경우 "경기 없음"으로 채우기
schedule_df.fillna("경기 없음", inplace=True)

# 결과 출력
display(schedule_df)

schedule_df.to_csv("schedule.csv", index=False)

,시간,여자 그룹 3_1,여자 그룹 3_2,여자 그룹 3_3,여자 그룹 3_4,여자 그룹 2_1,여자 그룹 2_2,여자 그룹 2_3,여자 그룹 2_4,여자 그룹 1_1,...,남자 그룹 3_3,남자 그룹 3_4,남자 그룹 2_1,남자 그룹 2_2,남자 그룹 2_3,남자 그룹 2_4,남자 그룹 1_1,남자 그룹 1_2,남자 그룹 1_3,남자 그룹 1_4
0,1,김지현,룰루,우당탕,Rent,뽐,사리,수임이,지니,가린,...,쿨쿨,터보,로센,매머드,안씨,페더러,성현,재근,Bdup,프리마
1,2,반이,크림,클레어,린다,민크,이피,제인,레이첼,기쁨,...,SJ,TG,브라운,영훈,전,푸마,매버릭,소니,정현,큐
2,3,룰루,우당탕,Rent,아롬,뽐,수임이,제시,파란하늘,가린,...,이언짱,제이슨,명자,미키찬,지호,Ace,비케이,성현,태보,토니
3,4,김지현,문문,야금,린다,젤로,이피,꼬맹,레이첼,리디아,...,쥬니혀니,터보,로센,페더러,태규,DY,동환,매버릭,밤송이,프리마
4,5,룰루,반이,크림,클레어,민크,제인,지니,파란하늘,꾸꾸,...,제이슨,쿨쿨,영훈,전,현민,Ace,성현,주방,큐,토니
5,6,김지현,야금,린다,아롬,뽐,강진,꼬맹,제시,기쁨,...,쥬니혀니,TG,매머드,미키찬,페더러,태규,매버릭,비케이,소니,정현
6,7,문문,우당탕,크림,Rent,민크,사리,수임이,파란하늘,다미,...,황더러,SJ,로센,명자,브라운,안씨,태보,Bdup,프리마,토니
7,8,김지현,반이,야금,아롬,젤로,강진,제시,지니,미아,...,제이슨,TG,미키찬,지호,페더러,푸마,동환,밤송이,성현,큐
8,9,룰루,문문,우당탕,클레어,제인,꼬맹,레이첼,파란하늘,가린,...,이언짱,터보,매머드,현민,Ace,DY,매버릭,주방,태보,프리마
9,10,반이,야금,크림,아롬,사리,강진,이피,지니,기쁨,...,쿨쿨,TG,안씨,영훈,푸마,태규,밤송이,비케이,소니,재근


## 개발코드 

In [98]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대 팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)와 같아야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        if all(games == rounds_target for games in player_games.values()) and len(matches) == len(players):
            return matches

# ===============================
# 2. 백트래킹 방식으로 G코트 leftover 배정
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 현재까지 배정된 G코트 결과 (길이 12, "경기 없음" 또는 경기 튜플)
    
    G코트에서는 인접 슬롯 조건은 제거하고, 오직 slot 자체에서 메인 스케줄 및 extra_assignment와 겹치지 않는지만 검사.
    """
    players_extra = get_players(extra_match)
    if get_players(main_schedule[slot]) & players_extra:
        return False
    if extra_assignment[slot] != "경기 없음":
        if get_players(extra_assignment[slot]) & players_extra:
            return False
    return True

def backtrack_assign_extra(extras, main_schedule, assignments, index, valid_slots):
    """
    extras: leftover 경기 리스트 (튜플들)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12)
    assignments: 현재까지 G코트 배정 결과 (길이 12, 초기값 "경기 없음")
    index: 현재 배정할 extras 리스트의 인덱스
    valid_slots: 배정 가능한 슬롯 리스트 (예: 인덱스 0~11 중, 7번 슬롯 제외)
    """
    if index == len(extras):
        return assignments  # 모든 경기 배정 완료
    current_match = extras[index]
    for slot in valid_slots:
        if assignments[slot] == "경기 없음" and can_place_extra_in_slot(current_match, slot, main_schedule, assignments):
            assignments[slot] = current_match
            result = backtrack_assign_extra(extras, main_schedule, assignments, index + 1, valid_slots)
            if result is not None:
                return result
            assignments[slot] = "경기 없음"  # 백트래킹
    return None

def assign_extra_schedule_for_group(extras, main_schedule):
    """
    extras: 리스트 leftover 경기 (튜플들)
    main_schedule: 메인 스케줄 (길이 12)
    백트래킹을 통해 extras를 12시간대 내 (단, 인덱스 7 제외) 배정.
    가능한 배정 결과(길이 12 리스트)를 반환, 없으면 None.
    """
    valid_slots = [i for i in range(12) if i != 7]  # 인덱스 7은 "이벤트 시간"
    assignments = ["경기 없음"] * 12
    return backtrack_assign_extra(extras, main_schedule, assignments, 0, valid_slots)

# ===============================
# 3. 최종 스케줄 생성 및 코트 배정 (시간대 1~12)
# ===============================

def main_schedule_assignment(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4):
    """
    player_groups: dict, 그룹 이름 -> 선수 리스트
    target_rounds: 사용 가능한 시간대 수 (여기서는 12)
    max_global_attempts: 글로벌 재시도 최대 횟수
    rounds_target: 각 선수가 해야 하는 경기 수 (여기서는 4)
    
    - 각 선수는 정확히 4경기를 진행
    - 각 그룹별로 생성된 경기 수 = 그룹 인원수
    - 메인 스케줄: 생성 경기 중 처음 target_rounds (1~12경기)
    - leftover(13경기 이상)는 G코트에 배정, 단 한 슬롯에 한 경기만 배정하며,
      우선순위 순으로 배정 (그리고 8번째 시간대(시간=8, 인덱스 7)는 "이벤트 시간"으로 처리)
    """
    # (가) 각 그룹별 경기 생성
    group_results = {}
    for group_name, players in player_groups.items():
        group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=rounds_target)
    
    # (나) 메인 스케줄과 leftover 분리
    main_schedules = {}
    extra_schedules = {}
    for group_name, matches in group_results.items():
        main_schedules[group_name] = matches[:target_rounds]   # 인덱스 0~11 => 시간대 1~12
        extra_schedules[group_name] = matches[target_rounds:]    # leftover 경기 (있으면)
    
    # (다) 각 그룹별로 leftover 경기 배정 (G코트 배정) - 백트래킹 사용
    group_extra_assignment = {}
    for group, extras in extra_schedules.items():
        assignment = assign_extra_schedule_for_group(extras, main_schedules[group])
        if assignment is None:
            return None  # 재시도 실패
        group_extra_assignment[group] = assignment
    
    # (라) 최종 G코트 스케줄: 각 시간대에 단 하나의 leftover 경기만 배정 (우선순위 순)
    # 우선순위: ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    final_G_schedule = ["경기 없음"] * target_rounds
    for slot in range(target_rounds):
        # 8번째 시간대 (인덱스 7)는 강제로 "이벤트 시간" 처리
        if slot == 7:
            final_G_schedule[slot] = "이벤트 시간"
            continue
        for group in group_order:
            extra_assign = group_extra_assignment.get(group, ["경기 없음"] * target_rounds)
            if extra_assign[slot] != "경기 없음":
                team1, team2 = extra_assign[slot]
                final_G_schedule[slot] = f"{', '.join(team1)} vs {', '.join(team2)}"
                break  # 한 슬롯에 한 경기만 배정
    
    # (마) A~F 코트 배정 (메인 스케줄)
    # 코트 매핑:
    # A코트: 여자 그룹 3, B코트: 여자 그룹 2, C코트: 여자 그룹 1,
    # D코트: 남자 그룹 3, E코트: 남자 그룹 2, F코트: 남자 그룹 1
    court_map = {
        "A코트": "여자 그룹 3",
        "B코트": "여자 그룹 2",
        "C코트": "여자 그룹 1",
        "D코트": "남자 그룹 3",
        "E코트": "남자 그룹 2",
        "F코트": "남자 그룹 1"
    }
    schedule_df = pd.DataFrame(columns=["시간"] + list(court_map.keys()) + ["G코트"])
    schedule_df["시간"] = range(1, target_rounds+1)
    
    for round_idx in range(target_rounds):
        for court, group in court_map.items():
            if group in main_schedules and round_idx < len(main_schedules[group]):
                match = main_schedules[group][round_idx]
                if match != "경기 없음":
                    team1, team2 = match
                    schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
                else:
                    schedule_df.at[round_idx, court] = "경기 없음"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]
    
    schedule_df.fillna("경기 없음", inplace=True)
    return schedule_df

# ===============================
# 4. 체크 함수: 각 선수 경기 수 및 동일 시간대 중복 검사
# ===============================

def parse_match(match_str):
    """
    match_str: "A vs B" 형태의 문자열 (또는 "경기 없음", "이벤트 시간")
    반환: 해당 경기에서 출전한 선수의 집합.
    """
    if match_str in ["경기 없음", "이벤트 시간"]:
        return set()
    try:
        teams = match_str.split(" vs ")
        team1 = [x.strip() for x in teams[0].split(",")]
        team2 = [x.strip() for x in teams[1].split(",")]
        return set(team1 + team2)
    except Exception as e:
        return set()

def count_games_per_player(schedule_df):
    """
    schedule_df: 최종 일정표 DataFrame (시간, A코트, B코트, ..., G코트)
    각 선수별 출전 횟수를 계산하여 dict로 반환.
    """
    counts = {}
    for idx, row in schedule_df.iterrows():
        for col in schedule_df.columns:
            if col == "시간":
                continue
            players = parse_match(row[col])
            for player in players:
                counts[player] = counts.get(player, 0) + 1
    return counts

def check_same_time_slot(schedule_df):
    """
    schedule_df: 최종 일정표 DataFrame (시간, A코트, ..., G코트)
    각 시간대(행)에서 여러 코트에 걸쳐 동일한 선수가 출전한 경우를 찾아 dict로 반환.
    반환 형식: {시간대: {선수: 중복 횟수, ...}, ...}
    """
    conflicts = {}
    for idx, row in schedule_df.iterrows():
        time_slot = row["시간"]
        all_players = []
        for col in schedule_df.columns:
            if col == "시간":
                continue
            all_players.extend(list(parse_match(row[col])))
        counter = {}
        for player in all_players:
            counter[player] = counter.get(player, 0) + 1
        slot_conflicts = {player: cnt for player, cnt in counter.items() if cnt > 1}
        if slot_conflicts:
            conflicts[time_slot] = slot_conflicts
    return conflicts

# ===============================
# 5. 글로벌 재시도: 모든 선수의 경기 수가 4회가 될 때까지 재생성
# ===============================

if __name__ == '__main__':
    # 예시 그룹 (원하는 대로 수정 가능)
    player_groups = {
        "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
        "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
        "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
        "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
        "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
        "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"]
    }
    
    max_global_attempts = 1000
    final_schedule_df = None
    for attempt in range(max_global_attempts):
        schedule_df = main_schedule_assignment(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4)
        if schedule_df is None:
            continue
        game_counts = count_games_per_player(schedule_df)
        # 모든 선수가 정확히 4경기인지 확인
        if all(count == 4 for count in game_counts.values()):
            final_schedule_df = schedule_df
            break

    if final_schedule_df is not None:
        display(final_schedule_df)
        print("\n각 선수별 출전 횟수:")
        game_counts = count_games_per_player(final_schedule_df)
        for player, cnt in sorted(game_counts.items()):
            print(f"{player}: {cnt}회")
        
        conflicts = check_same_time_slot(final_schedule_df)
        if conflicts:
            print("\n동일 시간대에 중복 출전한 선수:")
            for time_slot, conf in conflicts.items():
                conflict_str = ", ".join([f"{p}({cnt}회)" for p, cnt in conf.items()])
                print(f"시간대 {time_slot}: {conflict_str}")
        else:
            print("\n동일 시간대에 중복 출전한 선수는 없습니다.")
    else:
        print("충분한 충돌 없는 배정 결과를 얻지 못했습니다.")


KeyboardInterrupt: 

In [33]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)가 되어야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            # 추가 경기 전에는 각 선수 아직 4경기 미만이어야 함.
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        # 검증: 각 선수의 경기 수가 정확히 rounds_target인지 확인
        if all(games == rounds_target for games in player_games.values()):
            # 총 경기 수는 (#players) (각 경기 4명씩, 4경기씩이면 (#players)*4/4 = #players)
            if len(matches) == len(players):
                return matches
        # 실패 시 재시도

# ===============================
# 2. G코트 배정 로직 (leftover 경기 배정)
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 = available rounds, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 해당 그룹의 현재 G코트 배정 결과 (길이 = available rounds, "경기 없음" 또는 경기 튜플)
    
    slot과 인접 슬롯(slot-1, slot+1)에서 main_schedule 및 extra_assignment에 겹치는 선수가 없어야 함.
    """
    players_extra = get_players(extra_match)
    for s in [slot-1, slot, slot+1]:
        if 0 <= s < 12:
            if get_players(main_schedule[s]) & players_extra:
                return False
            if extra_assignment[s] != "경기 없음":
                if get_players(extra_assignment[s]) & players_extra:
                    return False
    return True

# ===============================
# 3. 최종 스케줄 생성 및 코트 배정 (시간대 1~12)
# ===============================

# 예시 선수 그룹들 (원래 데이터에 따라 그룹별 인원수는 다를 수 있음)
player_groups = {
    "여자 그룹 3": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
    "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
    "여자 그룹 1": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
    "남자 그룹 3": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"],
    "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
    "남자 그룹 1": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"]
}

random.seed(40)

# (가) 각 그룹별 경기 생성 – 각 선수 4경기, 즉 생성된 경기 수 = (#players)
group_results = {}
for group_name, players in player_groups.items():
    # max_matches를 해당 그룹 인원수로 설정
    group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=4)

# (나) 메인 스케줄(1~12경기)와 leftover(13경기 이상) 분리  
# 각 그룹별 생성 경기 수 = (#players).  
# 시간이 1~12만 있으므로, 메인 스케줄 = 첫 12경기, leftover = 나머지 (만약 #players > 12)
main_schedules = {}
extra_schedules = {}
for group_name, matches in group_results.items():
    main_schedules[group_name] = matches[:12]   # 1~12경기
    extra_schedules[group_name] = matches[12:]  # leftover 경기 (있으면)

# (다) 각 그룹별로 leftover 경기(있다면)를 12시간대 내 G코트에 배정  
# 각 그룹별 G코트 배정 결과: 길이 12 리스트, 초기값 "경기 없음"
group_extra_assignment = {}
for group, extras in extra_schedules.items():
    extra_assignment = ["경기 없음"] * 12
    for extra in extras:
        placed = False
        for slot in range(12):
            if extra_assignment[slot] == "경기 없음":
                if can_place_extra_in_slot(extra, slot, main_schedules[group], extra_assignment):
                    extra_assignment[slot] = extra
                    placed = True
                    break
        if not placed:
            print(f"[Warning] 그룹 {group}의 leftover 경기 {extra} 배정 실패 (충돌 발생)")
    group_extra_assignment[group] = extra_assignment

# (라) 최종 G코트 스케줄: 여러 그룹의 leftover 경기를 시간대별로 모아서 "|"로 결합  
final_G_schedule = [""] * 12
for slot in range(12):
    slot_matches = []
    for group, extra_assign in group_extra_assignment.items():
        if extra_assign[slot] != "경기 없음":
            team1, team2 = extra_assign[slot]
            slot_matches.append(f"{group}: {', '.join(team1)} vs {', '.join(team2)}")
    if slot_matches:
        final_G_schedule[slot] = " | ".join(slot_matches)
    else:
        final_G_schedule[slot] = "경기 없음"

# (마) A~F 코트 배정 (메인 스케줄)  
# 코트 매핑:
# A코트 (여자 그룹 3), B코트 (여자 그룹 2), C코트 (여자 그룹 1),
# D코트 (남자 그룹 3), E코트 (남자 그룹 2), F코트 (남자 그룹 1)
court_map = {
    "A코트": "여자 그룹 3",
    "B코트": "여자 그룹 2",
    "C코트": "여자 그룹 1",
    "D코트": "남자 그룹 3",
    "E코트": "남자 그룹 2",
    "F코트": "남자 그룹 1"
}
num_rounds = 12
columns = ["시간"] + list(court_map.keys()) + ["G코트"]
schedule_df = pd.DataFrame(columns=columns)
schedule_df["시간"] = range(1, num_rounds+1)

for round_idx in range(num_rounds):
    for court, group in court_map.items():
        if group in main_schedules and round_idx < len(main_schedules[group]):
            match = main_schedules[group][round_idx]
            if match != "경기 없음":
                team1, team2 = match
                schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        else:
            schedule_df.at[round_idx, court] = "경기 없음"
    schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]

schedule_df.fillna("경기 없음", inplace=True)
display(schedule_df)


[Warning] 그룹 여자 그룹 3의 leftover 경기 (('기쁨', '레일라'), ('미아', '연주')) 배정 실패 (충돌 발생)
[Warning] 그룹 여자 그룹 2의 leftover 경기 (('이피', '제인'), ('꼬맹', '파란하늘')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 3의 leftover 경기 (('매버릭', '성현'), ('소니', '토니')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 3의 leftover 경기 (('동환', '정현'), ('큐', '태보')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 2의 leftover 경기 (('매머드', '미키찬'), ('영훈', 'DY')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 2의 leftover 경기 (('브라운', '페더러'), ('현민', 'Ace')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 2의 leftover 경기 (('로센', '매머드'), ('전', 'DY')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 1의 leftover 경기 (('김군', '스테판'), ('제이슨', '쥬니혀니')) 배정 실패 (충돌 발생)
[Warning] 그룹 남자 그룹 1의 leftover 경기 (('마크', '성일월드'), ('이언짱', 'TG')) 배정 실패 (충돌 발생)


,시간,A코트,B코트,C코트,D코트,E코트,F코트,G코트
0,1,"리디아, 유하 vs 럽테닛, 연주","제인, 꼬맹 vs 제시, 파란하늘","김지현, 룰루 vs 반이, 린다","밤송이, 재근 vs Bdup, 토니","미키찬, 지호 vs Ace, 태규","마크, 스테판 vs 쥬니혀니, SJ",경기 없음
1,2,"가린, 다미 vs 미아, 밀리","젤로, 뽐 vs 강진, 지니","문문, 크림 vs 클레어, Rent","동환, 비케이 vs 태보, 프리마","명자, 브라운 vs 현민, DY","김군, 로저 vs 우디, TG",경기 없음
2,3,"기쁨, 꾸꾸 vs Hailey, 연주","사리, 수임이 vs 제인, 제시","김지현, 반이 vs 야금, 린다","매버릭, 소니 vs 재근, 정현","로센, 전 vs 지호, Ace","스테판, 쿨쿨 vs 터보, 황더러",경기 없음
3,4,"가린, 레일라 vs 밀리, 커피믹스","젤로, 이피 vs 꼬맹, 지니","문문, 우당탕 vs 크림, 클레어","성현, 주방 vs 태보, 토니","명자, 매머드 vs 안씨, 페더러","로저, 우디 vs SJ, TG",경기 없음
4,5,"리디아, 꾸꾸 vs 미아, 유하","뽐, 수임이 vs 강진, 레이첼","반이, Rent vs 린다, 아롬","밤송이, 비케이 vs 큐, Bdup","미키찬, 영훈 vs 푸마, 태규","이언짱, 제이슨 vs 쥬니혀니, 쿨쿨",경기 없음
5,6,"가린, 럽테닛 vs 커피믹스, 연주","민크, 사리 vs 이피, 파란하늘","룰루, 문문 vs 야금, 우당탕","동환, 성현 vs 재근, 주방","브라운, 안씨 vs 전, 페더러","김군, 성일월드 vs 우디, SJ",경기 없음
6,7,"리디아, 레일라 vs 밀리, Hailey","수임이, 제시 vs 지니, 레이첼","김지현, 클레어 vs Rent, 아롬","밤송이, 정현 vs 큐, 프리마","영훈, 지호 vs 현민, 태규","마크, 제이슨 vs 쿨쿨, 황더러",경기 없음
7,8,"가린, 기쁨 vs 다미, 럽테닛","젤로, 민크 vs 뽐, 사리","룰루, 우당탕 vs 크림, 린다","비케이, 성현 vs 소니, 재근","로센, 미키찬 vs 푸마, Ace","로저, 스테판 vs 터보, SJ",경기 없음
8,9,"꾸꾸, 레일라 vs 유하, Hailey","강진, 제인 vs 제시, 레이첼","김지현, 문문 vs 야금, 아롬","매버릭, 밤송이 vs 주방, 태보","명자, 페더러 vs 태규, DY","김군, 마크 vs 성일월드, 황더러",경기 없음
9,10,"다미, 미아 vs 럽테닛, 커피믹스","민크, 꼬맹 vs 지니, 파란하늘","룰루, 반이 vs 우당탕, Rent","비케이, Bdup vs 프리마, 토니","로센, 안씨 vs 지호, 푸마","우디, 이언짱 vs 쥬니혀니, 터보",경기 없음


In [34]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)와 같아야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        if all(games == rounds_target for games in player_games.values()) and len(matches) == len(players):
            return matches

# ===============================
# 2. leftover 경기(G코트) 배정을 위한 함수 (랜덤 재조정)
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 현재까지 배정된 G코트 결과 (길이 12, "경기 없음" 또는 경기 튜플)
    
    slot 및 인접 슬롯(slot-1, slot+1)에서 메인 경기 및 이미 배정된 extra 경기와 선수 겹침 없을 것.
    """
    players_extra = get_players(extra_match)
    for s in [slot-1, slot, slot+1]:
        if 0 <= s < 12:
            if get_players(main_schedule[s]) & players_extra:
                return False
            if extra_assignment[s] != "경기 없음":
                if get_players(extra_assignment[s]) & players_extra:
                    return False
    return True

def assign_extra_schedule_for_group(extras, main_schedule, max_attempts=100):
    """
    extras: 리스트 leftover 경기 (튜플들)
    main_schedule: 메인 스케줄 (길이 12)
    재시도하면서, extras를 12시간대 내에 conflict 없이 배정할 수 있는지 랜덤 재조정.
    모든 extras가 배정되거나(여러 개가 동시에 배정될 수 있으므로, extras 수만큼 배정된 슬롯 수 == len(extras))
    가능한 배정 결과(길이 12 리스트)를 반환하며, 불가능하면 None 반환.
    """
    for attempt in range(max_attempts):
        assignment = ["경기 없음"] * 12
        extras_shuffled = extras[:]  # 복사 후 재조정
        random.shuffle(extras_shuffled)
        # 시간 슬롯 순서도 랜덤으로 시도
        slots = list(range(12))
        random.shuffle(slots)
        assigned_count = 0
        for extra in extras_shuffled:
            placed = False
            for slot in slots:
                if assignment[slot] == "경기 없음":
                    if can_place_extra_in_slot(extra, slot, main_schedule, assignment):
                        assignment[slot] = extra
                        assigned_count += 1
                        placed = True
                        break
            if not placed:
                break  # 이 시도에서는 모든 extras 배정 실패
        if assigned_count == len(extras):
            return assignment
    return None

# ===============================
# 3. 전체 스케줄 생성 및 코트 배정 (시간대 1~12)
#    (A~F 코트는 메인 스케줄, G코트는 leftover 경기 배정)
# ===============================

# 예시 선수 그룹들 (여자/남자 3그룹씩)
player_groups = {
    "여자 그룹 3": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
    "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
    "여자 그룹 1": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
    "남자 그룹 3": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"],
    "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
    "남자 그룹 1": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"]
}

random.seed(40)

max_global_attempts = 1000
final_success = False

for global_attempt in range(max_global_attempts):
    # (가) 각 그룹별 경기 생성 – 각 선수 정확히 4경기 (총 경기 수 = 그룹 인원수)
    group_results = {}
    for group_name, players in player_groups.items():
        group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=4)
    
    # (나) 메인 스케줄(1~12경기)와 leftover(13경기 이상) 분리
    main_schedules = {}
    extra_schedules = {}
    for group_name, matches in group_results.items():
        main_schedules[group_name] = matches[:12]   # 인덱스 0~11 => 시간대 1~12
        extra_schedules[group_name] = matches[12:]  # leftover 경기 (존재하면)
    
    # (다) 각 그룹별로 leftover 경기 (있다면)를 12시간대 내에 배정 (랜덤 재조정)
    group_extra_assignment = {}
    all_assigned = True
    for group, extras in extra_schedules.items():
        assignment = assign_extra_schedule_for_group(extras, main_schedules[group], max_attempts=100)
        if assignment is None:
            all_assigned = False
            break
        group_extra_assignment[group] = assignment
    if not all_assigned:
        # 재시도
        continue
    
    # (라) 최종 G코트 스케줄: 여러 그룹의 leftover 경기를 시간대별로 모아서 "|"로 결합
    final_G_schedule = [""] * 12
    for slot in range(12):
        slot_matches = []
        for group, extra_assign in group_extra_assignment.items():
            if extra_assign[slot] != "경기 없음":
                team1, team2 = extra_assign[slot]
                slot_matches.append(f"{group}: {', '.join(team1)} vs {', '.join(team2)}")
        final_G_schedule[slot] = " | ".join(slot_matches) if slot_matches else "경기 없음"
    
    # (마) A~F 코트 배정 (메인 스케줄)
    # 코트 매핑:
    # A코트 (여자 그룹 3), B코트 (여자 그룹 2), C코트 (여자 그룹 1),
    # D코트 (남자 그룹 3), E코트 (남자 그룹 2), F코트 (남자 그룹 1)
    court_map = {
        "A코트": "여자 그룹 3",
        "B코트": "여자 그룹 2",
        "C코트": "여자 그룹 1",
        "D코트": "남자 그룹 3",
        "E코트": "남자 그룹 2",
        "F코트": "남자 그룹 1"
    }
    num_rounds = 12
    columns = ["시간"] + list(court_map.keys()) + ["G코트"]
    schedule_df = pd.DataFrame(columns=columns)
    schedule_df["시간"] = range(1, num_rounds+1)
    
    for round_idx in range(num_rounds):
        for court, group in court_map.items():
            if group in main_schedules and round_idx < len(main_schedules[group]):
                match = main_schedules[group][round_idx]
                if match != "경기 없음":
                    team1, team2 = match
                    schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
                else:
                    schedule_df.at[round_idx, court] = "경기 없음"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]
    
    schedule_df.fillna("경기 없음", inplace=True)
    final_success = True
    break

if final_success:
    print(schedule_df)
else:
    print("충분한 충돌 없는 배정 결과를 얻지 못했습니다.")


KeyboardInterrupt: 

In [55]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)와 같아야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        if all(games == rounds_target for games in player_games.values()) and len(matches) == len(players):
            return matches

# ===============================
# 2. leftover 경기(G코트) 배정을 위한 함수 (인접 슬롯 조건 제거)
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 현재까지 배정된 G코트 결과 (길이 12, "경기 없음" 또는 경기 튜플)
    
    [변경됨] 인접 슬롯 조건 제거 → 오직 slot에서만 메인 스케줄 및 extra_assignment와 충돌 없는지 검사.
    """
    players_extra = get_players(extra_match)
    if get_players(main_schedule[slot]) & players_extra:
        return False
    if extra_assignment[slot] != "경기 없음":
        if get_players(extra_assignment[slot]) & players_extra:
            return False
    return True

def assign_extra_schedule_for_group(extras, main_schedule, max_attempts=100):
    """
    extras: 리스트 leftover 경기 (튜플들)
    main_schedule: 메인 스케줄 (길이 12)
    재시도하면서, extras를 12시간대 내에 conflict 없이 배정할 수 있는지 랜덤 재조정.
    모든 extras가 배정되면 배정 결과(길이 12 리스트)를 반환, 불가능하면 None 반환.
    """
    for attempt in range(max_attempts):
        assignment = ["경기 없음"] * 12
        extras_shuffled = extras[:]  # 복사 후 재조정
        random.shuffle(extras_shuffled)
        slots = list(range(12))
        random.shuffle(slots)
        assigned_count = 0
        for extra in extras_shuffled:
            placed = False
            for slot in slots:
                if assignment[slot] == "경기 없음":
                    if can_place_extra_in_slot(extra, slot, main_schedule, assignment):
                        assignment[slot] = extra
                        assigned_count += 1
                        placed = True
                        break
            if not placed:
                break  # 이 시도에서는 모든 extras 배정 실패
        if assigned_count == len(extras):
            return assignment
    return None

# ===============================
# 3. 전체 스케줄 생성 및 코트 배정 (시간대 1~12)
# ===============================

# 예시 선수 그룹들 (여자/남자 3그룹씩)
player_groups = {
    "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
    "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
    "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
    "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
    "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
    "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"]
}

# random.seed(40)

max_global_attempts = 1000
final_success = False

for global_attempt in range(max_global_attempts):
    # (가) 각 그룹별 경기 생성 – 각 선수 정확히 4경기 (총 경기 수 = 그룹 인원수)
    group_results = {}
    for group_name, players in player_groups.items():
        group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=4)
    
    # (나) 메인 스케줄(1~12경기)와 leftover(13경기 이상) 분리
    main_schedules = {}
    extra_schedules = {}
    for group_name, matches in group_results.items():
        main_schedules[group_name] = matches[:12]   # 인덱스 0~11 => 시간대 1~12
        extra_schedules[group_name] = matches[12:]  # leftover 경기 (있으면)
    
    # (다) 각 그룹별로 leftover 경기(있다면)를 12시간대 내에 배정 (랜덤 재조정)
    group_extra_assignment = {}
    all_assigned = True
    for group, extras in extra_schedules.items():
        assignment = assign_extra_schedule_for_group(extras, main_schedules[group], max_attempts=100)
        if assignment is None:
            all_assigned = False
            break
        group_extra_assignment[group] = assignment
    if not all_assigned:
        continue  # 글로벌 재시도
    
    # (라) 최종 G코트 스케줄: 각 시간대에 단 하나의 leftover 경기만 배정 (우선순위에 따라)
    # 우선순위: ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    final_G_schedule = ["경기 없음"] * 12
    for slot in range(12):
        # 8번째 시간대 (시간=8, index 7)는 강제로 "경기 없음"
        if slot == 7:
            final_G_schedule[slot] = "경기 없음"
            continue
        for group in group_order:
            extra_assign = group_extra_assignment.get(group, ["경기 없음"] * 12)
            if extra_assign[slot] != "경기 없음":
                team1, team2 = extra_assign[slot]
                final_G_schedule[slot] = f"{group}: {', '.join(team1)} vs {', '.join(team2)}"
                break  # 한 슬롯에는 한 경기만 배정
    
    # (마) A~F 코트 배정 (메인 스케줄)
    # 코트 매핑:
    # A코트 (여자 그룹 3), B코트 (여자 그룹 2), C코트 (여자 그룹 1),
    # D코트 (남자 그룹 3), E코트 (남자 그룹 2), F코트 (남자 그룹 1)
    court_map = {
        "A코트": "여자 그룹 3",
        "B코트": "여자 그룹 2",
        "C코트": "여자 그룹 1",
        "D코트": "남자 그룹 3",
        "E코트": "남자 그룹 2",
        "F코트": "남자 그룹 1"
    }
    num_rounds = 12
    columns = ["시간"] + list(court_map.keys()) + ["G코트"]
    schedule_df = pd.DataFrame(columns=columns)
    schedule_df["시간"] = range(1, num_rounds+1)
    
    for round_idx in range(num_rounds):
        for court, group in court_map.items():
            if group in main_schedules and round_idx < len(main_schedules[group]):
                match = main_schedules[group][round_idx]
                if match != "경기 없음":
                    team1, team2 = match
                    schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
                else:
                    schedule_df.at[round_idx, court] = "경기 없음"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]
    
    schedule_df.fillna("경기 없음", inplace=True)
    final_success = True
    break

if final_success:
    display(schedule_df)
else:
    print("충분한 충돌 없는 배정 결과를 얻지 못했습니다.")


,시간,A코트,B코트,C코트,D코트,E코트,F코트,G코트
0,1,"룰루, 문문 vs 반이, 클레어","민크, 사리 vs 꼬맹, 레이첼","꾸꾸, 미아 vs 밀리, 커피믹스","성일월드, 스테판 vs 황더러, TG","미키찬, 푸마 vs Ace, 태규","매버릭, 정현 vs 주방, 프리마","남자 그룹 1: 동환, 밤송이 vs 성현, 토니"
1,2,"김지현, 야금 vs Rent, 린다","젤로, 제시 vs 지니, 파란하늘","가린, 다미 vs 럽테닛, 연주","우디, 쥬니혀니 vs 쿨쿨, SJ","명자, 브라운 vs 전, DY","소니, 재근 vs Bdup, 토니",경기 없음
2,3,"룰루, 우당탕 vs 크림, 아롬","뽐, 사리 vs 수임이, 이피","리디아, 꾸꾸 vs 미아, 유하","김군, 마크 vs 이언짱, 제이슨","안씨, 영훈 vs 지호, 현민","동환, 주방 vs 큐, 태보",경기 없음
3,4,"문문, 반이 vs 야금, 린다","강진, 제시 vs 레이첼, 파란하늘","기쁨, 럽테닛 vs Hailey, 연주","성일월드, 우디 vs 터보, SJ","매머드, 미키찬 vs 페더러, Ace","밤송이, 비케이 vs 성현, 정현",경기 없음
4,5,"김지현, 우당탕 vs 클레어, Rent","젤로, 사리 vs 제인, 꼬맹","리디아, 다미 vs 레일라, 유하","스테판, 제이슨 vs 쿨쿨, 황더러","지호, 푸마 vs 태규, DY","주방, 큐 vs 프리마, 토니","여자 그룹 2: 뽐, 수임이 vs 제시, 레이첼"
5,6,"룰루, 크림 vs 린다, 아롬","민크, 강진 vs 이피, 지니","꾸꾸, 밀리 vs 럽테닛, Hailey","로저, 성일월드 vs 우디, TG","로센, 영훈 vs 전, 페더러","비케이, 재근 vs 정현, 태보","여자 그룹 1: 기쁨, 다미 vs 유하, 커피믹스"
6,7,"김지현, 문문 vs 반이, 우당탕","뽐, 제인 vs 꼬맹, 파란하늘","리디아, 레일라 vs 미아, 커피믹스","마크, 스테판 vs 제이슨, 쿨쿨","매머드, 브라운 vs 푸마, DY","동환, 매버릭 vs 성현, Bdup","남자 그룹 3: 로저, 이언짱 vs 쥬니혀니, 황더러"
7,8,"크림, 클레어 vs Rent, 아롬","사리, 수임이 vs 강진, 이피","가린, 기쁨 vs 밀리, 럽테닛","김군, 로저 vs 성일월드, SJ","로센, 미키찬 vs 안씨, 지호","비케이, 소니 vs 주방, 태보",경기 없음
8,9,"룰루, 야금 vs 우당탕, 린다","젤로, 민크 vs 제인, 레이첼","꾸꾸, 다미 vs 커피믹스, 연주","마크, 쥬니혀니 vs 쿨쿨, 터보","명자, 페더러 vs 현민, 태규","매버릭, 밤송이 vs 재근, 프리마","남자 그룹 2: 미키찬, 브라운 vs 안씨, 전"
9,10,"김지현, 반이 vs 클레어, 아롬","뽐, 강진 vs 제시, 지니","가린, 레일라 vs 밀리, Hailey","김군, 이언짱 vs 제이슨, TG","매머드, 안씨 vs 푸마, Ace","동환, 성현 vs 큐, Bdup",경기 없음


In [72]:
import itertools
import random
import pandas as pd

# ===============================
# 1. 그룹 내 경기 생성 (각 선수 정확히 4경기)
# ===============================

def initialize_data(players):
    """선수 목록 및 초기 데이터 설정"""
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """같은 팀에서는 1번만 같이할 수 있도록 검사"""
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """상대팀끼리는 2번까지만 만날 수 있도록 검사"""
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """매치를 추가하고 기록 업데이트"""
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players, max_matches, rounds_target=4):
    """
    한 그룹 내에서 각 선수가 정확히 rounds_target(=4)경기를 하도록 경기 생성.
    생성된 경기 수는 (#players)와 같아야 함.
    """
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < rounds_target for player in combination) and
                can_play_together(team1, player_pairs) and
                can_play_together(team2, player_pairs) and
                can_play_against(team1, team2, opponent_pairs) and
                not any(p in last_match_players for p in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
            
            if all(games == rounds_target for games in player_games.values()):
                break
        
        if all(games == rounds_target for games in player_games.values()) and len(matches) == len(players):
            return matches

# ===============================
# 2. leftover 경기(G코트) 배정을 위한 함수 (인접 슬롯 조건 제거)
# ===============================

def get_players(match):
    """경기(match)가 튜플이면 포함 선수 집합 반환, 아니면 빈 집합"""
    if match == "경기 없음":
        return set()
    team1, team2 = match
    return set(team1 + team2)

def can_place_extra_in_slot(extra_match, slot, main_schedule, extra_assignment):
    """
    extra_match: (team1, team2) 튜플
    slot: 제안하는 시간대 (인덱스 0~11)
    main_schedule: 해당 그룹의 메인 스케줄 (길이 12, 각 원소는 경기 튜플 또는 "경기 없음")
    extra_assignment: 현재까지 배정된 G코트 결과 (길이 12, "경기 없음" 또는 경기 튜플)
    
    인접 슬롯 조건은 제거하고, 오직 slot 자체에서 메인 스케줄 및 extra_assignment와 충돌이 없는지 검사.
    """
    players_extra = get_players(extra_match)
    if get_players(main_schedule[slot]) & players_extra:
        return False
    if extra_assignment[slot] != "경기 없음":
        if get_players(extra_assignment[slot]) & players_extra:
            return False
    return True

def assign_extra_schedule_for_group(extras, main_schedule, max_attempts=100):
    """
    extras: 리스트 leftover 경기 (튜플들)
    main_schedule: 메인 스케줄 (길이 12)
    재시도하면서, extras를 12시간대 내에 conflict 없이 배정할 수 있는지 랜덤 재조정.
    모든 extras가 배정되면 배정 결과(길이 12 리스트)를 반환, 불가능하면 None 반환.
    """
    for attempt in range(max_attempts):
        assignment = ["경기 없음"] * 12
        extras_shuffled = extras[:]  # 복사 후 재조정
        random.shuffle(extras_shuffled)
        slots = list(range(12))
        random.shuffle(slots)
        assigned_count = 0
        for extra in extras_shuffled:
            placed = False
            for slot in slots:
                if assignment[slot] == "경기 없음":
                    if can_place_extra_in_slot(extra, slot, main_schedule, assignment):
                        assignment[slot] = extra
                        assigned_count += 1
                        placed = True
                        break
            if not placed:
                break  # 이 시도에서는 모든 extras 배정 실패
        if assigned_count == len(extras):
            return assignment
    return None

# ===============================
# 3. 전체 스케줄 생성 및 코트 배정 (시간대 1~12)
# ===============================

def main(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4):
    """
    player_groups: dict, 그룹 이름 -> 선수 리스트
    target_rounds: 사용 가능한 시간대 수 (여기서는 12)
    max_global_attempts: 글로벌 재시도 최대 횟수
    rounds_target: 각 선수가 해야 하는 경기 수 (여기서는 4)
    
    조건:
      - 각 선수는 정확히 rounds_target 경기 (4경기)를 진행
      - 각 그룹별로 생성된 경기 수 = 그룹 인원수
      - 메인 스케줄: 생성 경기 중 처음 target_rounds (1~12경기)
      - leftover(13경기 이상)는 G코트에 배정, 단 한 슬롯에 한 경기만 배정하며,
        우선순위 순으로 배정 (그리고 9번째 시간대(인덱스 8)는 강제로 "경기 없음")
    """
    # (가) 각 그룹별 경기 생성 – 각 선수 정확히 rounds_target경기 (총 경기 수 = 그룹 인원수)
    group_results = {}
    for group_name, players in player_groups.items():
        group_results[group_name] = generate_schedule(players, max_matches=len(players), rounds_target=rounds_target)
    
    # (나) 메인 스케줄(1~target_rounds 경기)와 leftover(그 이후 경기) 분리
    main_schedules = {}
    extra_schedules = {}
    for group_name, matches in group_results.items():
        main_schedules[group_name] = matches[:target_rounds]   # 인덱스 0~11 => 시간대 1~12
        extra_schedules[group_name] = matches[target_rounds:]    # leftover 경기 (있으면)
    
    # (다) 각 그룹별로 leftover 경기(있다면)를 target_rounds 시간대 내에 배정 (랜덤 재조정)
    group_extra_assignment = {}
    all_assigned = True
    for group, extras in extra_schedules.items():
        assignment = assign_extra_schedule_for_group(extras, main_schedules[group], max_attempts=100)
        if assignment is None:
            all_assigned = False
            break
        group_extra_assignment[group] = assignment
    if not all_assigned:
        return None  # 글로벌 재시도 실패
    
    # (라) 최종 G코트 스케줄: 각 시간대에 단 하나의 leftover 경기만 배정 (우선순위 순)
    # 우선순위: ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]
    final_G_schedule = ["경기 없음"] * target_rounds
    for slot in range(target_rounds):
        # 8번째 시간대 (시간=8, 인덱스 7)는 강제로 "경기 없음"
        if slot == 7:
            final_G_schedule[slot] = "이벤트 시간"
            continue
        for group in group_order:
            extra_assign = group_extra_assignment.get(group, ["경기 없음"] * target_rounds)
            if extra_assign[slot] != "경기 없음":
                team1, team2 = extra_assign[slot]
                final_G_schedule[slot] = f"{group}: {', '.join(team1)} vs {', '.join(team2)}"
                break  # 한 슬롯에는 한 경기만 배정
    
    # (마) A~F 코트 배정 (메인 스케줄)
    # 코트 매핑:
    # A코트 (여자 그룹 3), B코트 (여자 그룹 2), C코트 (여자 그룹 1),
    # D코트 (남자 그룹 3), E코트 (남자 그룹 2), F코트 (남자 그룹 1)
    court_map = {
        "A코트": "여자 그룹 3",
        "B코트": "여자 그룹 2",
        "C코트": "여자 그룹 1",
        "D코트": "남자 그룹 3",
        "E코트": "남자 그룹 2",
        "F코트": "남자 그룹 1"
    }
    schedule_df = pd.DataFrame(columns=["시간"] + list(court_map.keys()) + ["G코트"])
    schedule_df["시간"] = range(1, target_rounds+1)
    
    for round_idx in range(target_rounds):
        for court, group in court_map.items():
            if group in main_schedules and round_idx < len(main_schedules[group]):
                match = main_schedules[group][round_idx]
                if match != "경기 없음":
                    team1, team2 = match
                    schedule_df.at[round_idx, court] = f"{', '.join(team1)} vs {', '.join(team2)}"
                else:
                    schedule_df.at[round_idx, court] = "경기 없음"
            else:
                schedule_df.at[round_idx, court] = "경기 없음"
        schedule_df.at[round_idx, "G코트"] = final_G_schedule[round_idx]
    
    schedule_df.fillna("경기 없음", inplace=True)
    return schedule_df

if __name__ == '__main__':
    
    # random.seed(40)
    
    # 예시 그룹 (원하는 대로 수정 가능)
    player_groups = {
        "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"],
        "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
        "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
        "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
        "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
        "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"]
    }
    
    schedule_df = main(player_groups, target_rounds=12, max_global_attempts=1000, rounds_target=4)
    if schedule_df is not None:
        display(schedule_df)
    else:
        print("충분한 충돌 없는 배정 결과를 얻지 못했습니다.")


,시간,A코트,B코트,C코트,D코트,E코트,F코트,G코트
0,1,"룰루, 야금 vs 크림, Rent","젤로, 뽐 vs 강진, 파란하늘","다미, 레일라 vs 럽테닛, 연주","김군, 성일월드 vs 제이슨, 황더러","로센, 미키찬 vs 푸마, 태규","밤송이, 소니 vs 정현, 큐",경기 없음
1,2,"김지현, 우당탕 vs 클레어, 아롬","민크, 이피 vs 제인, 제시","기쁨, 리디아 vs 미아, 커피믹스","로저, 이언짱 vs 쿨쿨, 터보","명자, 영훈 vs 지호, Ace","비케이, 재근 vs Bdup, 토니",경기 없음
2,3,"룰루, 문문 vs 반이, 린다","젤로, 수임이 vs 꼬맹, 레이첼","꾸꾸, 다미 vs 밀리, Hailey","마크, 쥬니혀니 vs SJ, TG","브라운, 페더러 vs 푸마, 현민","정현, 주방 vs 태보, 프리마","여자 그룹 1: 가린, 기쁨 vs 미아, 럽테닛"
3,4,"김지현, 야금 vs 크림, 아롬","민크, 뽐 vs 사리, 파란하늘","가린, 레일라 vs 유하, 커피믹스","성일월드, 스테판 vs 제이슨, 쿨쿨","매머드, 안씨 vs 전, 태규","동환, 소니 vs 큐, 토니","남자 그룹 2: 명자, 지호 vs Ace, DY"
4,5,"문문, 반이 vs 우당탕, 린다","젤로, 강진 vs 꼬맹, 지니","기쁨, 꾸꾸 vs 밀리, 럽테닛","김군, 우디 vs 쥬니혀니, TG","미키찬, 영훈 vs 현민, DY","매버릭, 성현 vs 정현, 프리마","남자 그룹 3: 이언짱, 제이슨 vs 쿨쿨, SJ"
5,6,"야금, 클레어 vs Rent, 아롬","사리, 제시 vs 레이첼, 파란하늘","리디아, 레일라 vs Hailey, 연주","마크, 스테판 vs 터보, SJ","매머드, 브라운 vs 페더러, 태규","동환, 밤송이 vs 재근, Bdup",경기 없음
6,7,"김지현, 룰루 vs 반이, 우당탕","뽐, 수임이 vs 제인, 지니","기쁨, 미아 vs 밀리, 유하","로저, 성일월드 vs 이언짱, 쥬니혀니","로센, 지호 vs 푸마, Ace","매버릭, 소니 vs 태보, 토니","남자 그룹 2: 안씨, 전 vs 페더러, 현민"
7,8,"문문, 크림 vs 클레어, Rent","민크, 강진 vs 이피, 파란하늘","가린, 리디아 vs 다미, 연주","스테판, 쿨쿨 vs 터보, TG","명자, 매머드 vs 영훈, 페더러","성현, 주방 vs 큐, Bdup",이벤트 시간
8,9,"룰루, 우당탕 vs 린다, 아롬","젤로, 사리 vs 수임이, 제인","유하, 럽테닛 vs 커피믹스, Hailey","우디, 쥬니혀니 vs 황더러, SJ","미키찬, 전 vs 태규, DY","밤송이, 비케이 vs 소니, 정현","남자 그룹 1: 동환, 성현 vs 주방, 태보"
9,10,"김지현, 문문 vs 반이, Rent","민크, 꼬맹 vs 제시, 지니","가린, 꾸꾸 vs 레일라, 연주","김군, 스테판 vs 제이슨, TG","로센, 브라운 vs 안씨, Ace","성현, 재근 vs 프리마, 토니",경기 없음
